In [75]:
# 主要ライブラリのimport
import pandas as pd
import numpy as np

#ファイル出力時にパスを指定する際によく使う
import os
import pathlib

# ailab_tools → ailabで作っている各種ツール群
# impala、GSheet、GMail等の操作が簡単にできる
# ドキュメントは下記
# https://s3-ap-northeast-1.amazonaws.com/ailab-smn-valis/ailab-tools/docs/html/index.html

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

#グラフをnotebook上で描画するためのおまじない
%matplotlib inline

In [76]:
# Impala接続情報 hc5
ircfg = {
    "hosts": ['172.16.60.117'],
    "port": 21050,
    "user": 'vmspool',
    "request_pool": 'adhoc_dm01_pool',
    "httpfs_host": '172.16.60.42',
    "httpfs_port": 14000
}

## B.logicad 上位100ドメインの各SSP別の配信状況、効果

### B0.該当月の上位ドメインを確認

In [77]:
start_at = "2025-05-01"
end_at = "2025-05-31"

In [78]:
# CMAテストにnet_salesが入らないケースがあり、original_winのTop100に変更（2024/03分から）

query = (
f"""
select 
concat_ws('-', year, month) ym,
domain_id,
domain,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2) ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
left join console.ssp s on ads.child_ssp_id=s.id
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and child_ssp_id>0 and domain_id>0
group by 1,2,3
order by 8 desc limit 100
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_0_1 = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [79]:
# path定義
parent_path = pathlib.Path("__file__").resolve().parent
file_path = parent_path / "data"/ f"SSP_domain_top100_{start_at}.tsv"

data_b_0_1.to_csv(file_path, index=False, sep="\t")

In [80]:
# 全体データ抽出
query = (
f"""
select 
concat_ws('-', year, month) ym,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2) ctv2,
	sum(original_win_price)/1000000 original_win,
---sum((1+csm.margin/100)*win_price)/1000000 net_sales_old,
---sum((1+csm.margin/100)/(1-cam.margin/100)*win_price)/1000000 gross_sales_old,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
left join console.ssp s on ads.child_ssp_id=s.id
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and child_ssp_id>0
group by 1
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_0_2 = ir.sql_to_pandas(query)
data_b_0_2

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

,ym,imp,click,ctv1,ctv2,original_win,net_sales,gross_sales
0,2025-05,3399040468,11941167,1625123,38622,261413183.789218000000000000,509200047.809538109009126742,640652289.998396224224494221


In [81]:
top100_domain_id="1378672,1400731,1333256,1343600,11958,2610833,1438261,14905,2860984,17762,15668,1166771,14143,19653,3235804,1296493,1301738,3636870,3624873,2347666,3366040,3509684,1858733,13166,1400732,3094367,1292793,2641332,1400736,1400733,1346931,1421179,3450218,2144804,48687,15199,1431274,1540746,16502,12648,1431740,3549833,3577859,1400740,1443067,17542,1341826,1302801,1359120,12641,3510773,1438244,20410,1328390,3020678,1382773,1325982,2054036,2931541,1330767,1400730,2686800,3423651,3408452,17589,1969039,1433550,12239,2031045,10252,16650,13689,1441616,16398,11829,1518228,14909,3759271,3742980,13618,1930765,3213054,11469,14486,3502468,1452626,1969040,19694,1459838,2713329,13588,15253,13535,3382526,1409769,1377359,2507017,1296559,1906330,19389"

### B1.共通案件確認

In [82]:
# 除外リスト（広告主✖️SSP✖️OS(is_app)でCTV1が1000を超えるものを対象）
exclude_ads_id = "23626,24377,8139,24580,15701,23336,16763,8426,5930"

In [83]:
query = (
f"""
select 
concat_ws('-', year, month) ym,
ads.child_ssp_id,
s.name ssp,
    ads.advertiser_id,
    advertiser_name,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'IB_video'else 'display' end creative,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2) ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum( gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
left join console.ssp s on ads.child_ssp_id=s.id
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and ads.advertiser_id not in({exclude_ads_id}) -- ブラックリストは除外

and child_ssp_id>0
and domain_id in ({top100_domain_id})
group by 1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_1_1 = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [84]:
# path定義
parent_path = pathlib.Path("__file__").resolve().parent
file_path = parent_path / "data"/ f"SSP_domain_top100_{start_at}_adv.tsv"

data_b_1_1.to_csv(file_path, index=False, sep="\t")

### B.共通広告主でのレポート

In [85]:
pc_display_advertiser_id="25342,22747,24355,25321,25334,25438,24962,23278,24652,22903,46,25228,23786,24624,23711,7694,7783,18937,25450,22625,25469,24126,22693,24217,23704,22532,19242,19552,24866,20870,7151,24969,9324,25434"
pc_native_advertiser_id="23278,22747,5900,19552,11092,22233,20870,16006,22672,24632,23327,14398,7294,20909,23786,19640,22532,13696,25093"
pc_ibv_advertiser_id="24296,25491,23161"
spweb_display_advertiser_id="17989,10847,23191,25342,25321,17278,23278,24624,23437,19539,4264,25213,24487,24237,24959,12766,7983,23028,24828,23414,23155,21736,24583,23711,24962,23960,23202,24330,24608,21588,19574,23704,22747,23844,23406,23435,20870,4576,13471,23786,16483,16737,25228,23194,19171,23180,9700,18989,25334,24245,24813,19679,23419,24627,20909,24359,24961,12597,20974,23275,25035,22069,24922,25229,25259,23253,24529,24126,22532,22631,9662,25438,17800,22693,25311,19489,23282,24553,13837,24687,23980,25200,18033,23843,18937,25022,21986,21495,24986,21765,25269,24814,21985,22316,24355,25109,15838,19052,22558,19694,24598,25427,24652,18110,23878,24931,22672,20746,25302,24074,24832,25316,25450,21367,24969,24717,24963,25514,53,19951,24316,23059,24253,22191,23698,24632,22394,10919,3629,24499,10156,24911,19544,22524,22466,6358,25236,25198,8595,12345,1782,19033,24101,18880,25249,20904,18792,25506,25008,17243,9324,25219,19552,20832,17424,5374,5351,7078,25120,23628,7294,16756,10632,23038,25422,14398,24022,23903,2763,13104,4996,25386,20968,23372,17579"
spweb_native_advertiser_id="23191,22233,24627,18861,23437,20870,24624,4264,20909,21736,7983,23903,17646,22558,24330,23028,24959,24499,20974,23315,19906,23443,25626,13471,22532,18294,25256,22524,19481,17579,22672,22394,23843,18989,22069,20904,21588"
spweb_ibv_advertiser_id="25446,25414,22964,24296,23161,24168,24027,25286"
spapp_display_advertiser_id="23155,15838,24245,25438,23704,23194,13471,23711,20870,22672,20909,22532,24126,18110,13837,19574,23419,25450,22693,24499,23059,21495"
spapp_native_advertiser_id="18754,23437,24330,22625,22558,23960,25269,9662,24959,21736,20892,23202,15941,19693,24624,24608,24069,25242,23315,24961,20974,22233,24832,25626,18989,25464,15838,23435,22394,24922,23028,24499,19679,24627,22532,24621,17646,20746,23191,22672,25438,13471,20909,20870,16334,23695,23711,24911,23786,24328,22693,24217,24606,25452,25022,3066,25450,25088,24828,25422,17579,24156,25200,24899,22912,23878,18600,22926,24359,24019,25267,23277,4264,25427,10847,22012,24218,25514,24006,24126,22274,25392,24896,24016"

In [86]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10,11,13)
and domain_id in ({top100_domain_id})
and a.advertiser_id in({pc_display_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10,11,13)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({pc_display_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_pc_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [87]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and domain_id in({top100_domain_id})
and a.advertiser_id in({pc_native_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({pc_native_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_pc_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [88]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(11,13)
and domain_id in({top100_domain_id})
and a.advertiser_id in({pc_ibv_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family not in('iOS','Android')
and is_app=false
and creative_type in(11,13)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({pc_ibv_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_pc_ib = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [89]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10,11,13)
and domain_id in({top100_domain_id})
and a.advertiser_id in({spweb_display_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type not in(5,6,10,11,13)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({spweb_display_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_spweb_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [90]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and domain_id in({top100_domain_id})
and a.advertiser_id in({spweb_native_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(5,6)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({spweb_native_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_spweb_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [91]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(11,13)
and domain_id in({top100_domain_id})
and a.advertiser_id in({spweb_ibv_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=false
and creative_type in(11,13)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({spweb_ibv_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_spweb_ib = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [92]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type not in(5,6,10,11,13)
and domain_id in({top100_domain_id})
and a.advertiser_id in({spapp_display_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type not in(5,6,10,11,13)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({spapp_display_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_spapp_display = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [93]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type in(5,6)
and domain_id in({top100_domain_id})
and a.advertiser_id in({spapp_native_advertiser_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and user_agent_os_family in('iOS','Android')
and is_app=true
and creative_type in(5,6)
and domain_id in({top100_domain_id})
and ads.advertiser_id in({spapp_native_advertiser_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_spapp_native = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [94]:
## default
data_b=pd.concat([data_b_pc_display,data_b_pc_native,data_b_pc_ib,data_b_spweb_display,data_b_spweb_native,data_b_spweb_ib,data_b_spapp_display,data_b_spapp_native])

In [95]:
# path定義
parent_path = pathlib.Path("__file__").resolve().parent
file_path = parent_path / "data"/ f"SSP_domain_top100_{start_at}_sum_adv.tsv"

data_b.to_csv(file_path, index=False, sep="\t")

### B.全案件の上位ドメイン別

In [96]:
# f-string という記法を使うと変数の挿入が簡単にできる
query = (
f"""
select 
ym,
    os,
    is_app,
    creative,
	child_ssp_id,
	s.name ssp,
domain,
case
when hie.private_brand_id = 107 then "7:TVBridge"
when hie.ad_type =5 then '6:DOOH'
when hie.oem_id=300 then '5:CPX'
when summary_name = "RTG" then "1:RTG"
when summary_name = "DyC" then "2:DyC"
when summary_name = "類似" then "3:類似拡張"
when summary_name in ("ATA","カスタム(ブロード)","動画","カスタムATA") then "4:オーディエンス(動画含)"
else "9:その他" end as category,
	sum(res) res,
	sum(bidden)bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win) original_win,
	sum(net_sales) net_sales,
	sum(gross_sales) gross_sales
from
(
select
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
    a.target_id,
	sum(res) res,
	sum(case when bidden=true then res else 0 end) bidden,
	0 imp,
	0 click,
	0 ctv1,
	0 ctv2,
	0 original_win,
    0 net_sales,
    0 gross_sales
from dm.domain_res a
inner join dm.hierarchies hie using(target_id)
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and child_ssp_id>0
and domain_id in({top100_domain_id})
group by 1,2,3,4,5,6,7

union all

select 
concat_ws('-', year, month) ym,
    case when user_agent_os_family in('iOS','Android') then 'SP' else 'PC' end os,
    case when is_app is null then 0 else is_app end is_app,
    case when creative_type in(5,6) then 'native' when creative_type in(10) then 'vast_video' when creative_type in(11,13) then 'ib_video' else 'display' end creative,
	child_ssp_id,
    domain,
ads.target_id,
	0 res,
	0 bidden,
	sum(imp) imp,
	sum(click)click,
	sum(ctv1) ctv1,
	sum(ctv2)ctv2,
	sum(original_win_price)/1000000 original_win,
    sum(net_spend)/1000000 net_sales,
    sum(gross_spend)/1000000 gross_sales
from dm.domain_ads ads
inner join dm.hierarchies hie on ads.target_id = hie.target_id
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date
where concat_ws('-', year, month, day) between '{start_at}' and '{end_at}'
and child_ssp_id>0
and domain_id in({top100_domain_id})
group by 1,2,3,4,5,6,7) a
left join console.ssp s on s.id=a.child_ssp_id
inner join dm.product_target_master c on c.target_id=a.target_id
inner join dm.hierarchies hie on a.target_id = hie.target_id
group by  1,2,3,4,5,6,7,8
"""
)
with ImpalaResource(**ircfg) as ir:
    data_b_all = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [97]:
# path定義
parent_path = pathlib.Path("__file__").resolve().parent
file_path = parent_path / "data"/ f"SSP_domain_top100_{start_at}_all_adv.tsv"

data_b_all.to_csv(file_path, index=False, sep="\t")